In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
a = glob.glob('./features/*.txt')

In [3]:
features={}

In [4]:
for name in a:
    try:
        with open(name) as f:
            # read in the data 
            temp_df = pd.read_csv(name,delim_whitespace=True,header=None)
            # make the row names cycle as a variable
            temp_df.index.name = 'cycle'
            temp_df.reset_index(inplace=True)
            # transpose the data 
            temp_df_transposed= temp_df.T
            # make the seconds a variable, call it "time”
            temp_df_transposed.index.name = 'time'
            temp_df_transposed.reset_index(inplace=True)
            # add a prefix cycle in the column names to help with pivoting data (from wide to long)
            string = ' cycle'.join(str(e) for e in list(temp_df_transposed.columns))
            temp_df_transposed.columns = string.split(" ")
            # From wide to long to help with joining all the variables 
            temp_df_long = pd.wide_to_long(temp_df_transposed.iloc[1:,:],stubnames='cycle', i=['time'], j='c')
            temp_df_long.reset_index(inplace=True)
            # save each data variable in the long format into separate data frames in the dictionary called "features"
            # but exclude path and txt in the names
            features[name[9:-4]] = temp_df_long
                        
           
    
    # prevent a possible error
    except IOError as exc:
        if exc.errno != errno.EISDIR:
            raise

In [5]:
features.keys()

dict_keys(['s/CE', 's/CP', 's/EPS1', 's/FS1', 's/FS2', 's/PS1', 's/PS2', 's/PS3', 's/PS4', 's/PS5', 's/PS6', 's/SE', 's/TS1', 's/TS2', 's/TS3', 's/TS4', 's/VS1'])

In [6]:
for key in list(features.keys()):
    features[key].columns=['seconds','cycle',key]

In [7]:
dfs= [features['s/CP'],
      features['s/CE'],
      features['s/EPS1'],
      features['s/FS1'],
      features['s/FS2'],
      features['s/PS1'],
      features['s/PS2'],
      features['s/PS3'],
      features['s/PS4'],
      features['s/PS5'],
      features['s/PS6'],
      features['s/SE'],
      features['s/TS1'],
      features['s/TS2'],
      features['s/TS3'],
      features['s/TS4'],
     features['s/VS1']]

In [8]:
dfs

[        seconds  cycle   s/CP
 0             0      0  2.184
 1             1      0  2.184
 2             2      0  2.184
 3             3      0  2.185
 4             4      0  2.178
 ...         ...    ...    ...
 132295       55   2204  2.134
 132296       56   2204  2.146
 132297       57   2204  2.144
 132298       58   2204  2.136
 132299       59   2204  2.148
 
 [132300 rows x 3 columns],
         seconds  cycle    s/CE
 0             0      0  47.202
 1             1      0  47.273
 2             2      0  47.250
 3             3      0  47.332
 4             4      0  47.213
 ...         ...    ...     ...
 132295       55   2204  46.355
 132296       56   2204  46.432
 132297       57   2204  46.384
 132298       58   2204  46.479
 132299       59   2204  46.621
 
 [132300 rows x 3 columns],
           seconds  cycle  s/EPS1
 0               0      0  2411.6
 1               1      0  2411.6
 2               2      0  2411.6
 3               3      0  2411.6
 4            

In [9]:
from functools import reduce
feats_join = reduce(lambda x,y: pd.merge(x,y,on=['seconds','cycle']),dfs)

In [10]:
feats_join.head()

,seconds,cycle,s/CP,s/CE,s/EPS1,s/FS1,s/FS2,s/PS1,s/PS2,s/PS3,s/PS4,s/PS5,s/PS6,s/SE,s/TS1,s/TS2,s/TS3,s/TS4,s/VS1
0,0,0,2.184,47.202,2411.6,8.990,10.179,151.47,125.50,2.305,0.0,9.936,9.818,68.039,35.570,40.961,38.320,30.363,0.604
1,1,0,2.184,47.273,2411.6,0.770,10.174,151.45,125.39,2.305,0.0,9.947,9.823,0.000,35.492,40.949,38.332,30.375,0.605
2,2,0,2.184,47.250,2411.6,0.641,10.151,151.52,125.40,2.336,0.0,9.964,9.844,0.000,35.469,40.965,38.320,30.367,0.611
3,3,0,2.185,47.332,2411.6,0.006,10.149,151.27,125.03,2.578,0.0,9.989,9.861,0.000,35.422,40.922,38.324,30.367,0.603
4,4,0,2.178,47.213,2411.6,0.000,10.172,150.80,124.05,2.977,0.0,9.996,9.877,0.000,35.414,40.879,38.332,30.379,0.608


In [43]:
label = pd.read_csv('profile.txt')
label.columns = ['cooler_condition', 'valve_condition', 'pump_leak', 'hydraulic_accumulator', 'stable_flag']

ValueError: Length mismatch: Expected axis has 1 elements, new values have 5 elements

In [42]:
label

,3\t100\t0\t130\t1
0,3\t100\t0\t130\t1
1,3\t100\t0\t130\t1
2,3\t100\t0\t130\t1
3,3\t100\t0\t130\t1
4,3\t100\t0\t130\t1
...,...
2199,100\t100\t0\t90\t0
2200,100\t100\t0\t90\t0
2201,100\t100\t0\t90\t0
2202,100\t100\t0\t90\t0


In [11]:
label = pd.read_csv('profile.txt',delim_whitespace=True)
label.columns = ['cooler_condition', 'valve_condition', 'pump_leak', 'hydraulic_accumulator', 'stable_flag']

In [12]:
%pip install tsfresh

     |████████████████████████████████| 91 kB 4.4 MB/s eta 0:00:011
     |████████████████████████████████| 669 kB 10.5 MB/s eta 0:00:01
     |████████████████████████████████| 884 kB 13.7 MB/s eta 0:00:01
     |████████████████████████████████| 55 kB 4.8 MB/s  eta 0:00:01
     |████████████████████████████████| 98 kB 5.5 MB/s  eta 0:00:01
  Created wheel for locket: filename=locket-0.2.0-py3-none-any.whl size=4040 sha256=a6d61df8b9963860a83fb40af009c73b874d6eb9560c4fae65c02db9224b60fd
  Stored in directory: /home/azureuser/.cache/pip/wheels/21/ca/95/1e41f9a9a7a06ba06874ad2bf0eac88e7ba02fe9a6ff26c77d
Successfully built locket
Note: you may need to restart the kernel to use updated packages.


In [13]:
import xgboost
from xgboost import XGBClassifier, XGBRegressor

xgc = XGBClassifier()
xgr = XGBRegressor()


from tsfresh.transformers import RelevantFeatureAugmenter
from sklearn.pipeline import Pipeline

import joblib

In [14]:
def preprocess_accum(ddf):
    ddf.index.name = 'cycle'
    ddf = pd.DataFrame(ddf)
    ddf.reset_index(inplace=True)
    # transpose the data 
    ddf_transposed= ddf.T
    # make the seconds a variable, call it "time”
    ddf_transposed.index.name = 'time'
    ddf_transposed.reset_index(inplace=True)
    # add a prefix cycle in the column names to help with pivoting data (from wide to long)
    string = ' cycle'.join(str(e) for e in list(ddf_transposed.columns))
    ddf_transposed.columns = string.split(" ")
    # From wide to long to help with joining all the variables 
    ddf_long = pd.wide_to_long(ddf_transposed.iloc[1:,:],stubnames='cycle', i=['time'], j='c')
    ddf_long.reset_index(inplace=True)
    # save each data variable in the long format into separate data frames in the dictionary called "features"
    # but exclude path and txt in the names
    features[name[9:-4]] = ddf_long   
    # prevent a possible error
    for key in list(features.keys()):
        features[key].columns=['seconds','cycle',key]
        
       
    return features

In [49]:
#  I suspect the function above to be useless.

In [15]:
pipeline = Pipeline([('augmenter', RelevantFeatureAugmenter(column_id="cycle", column_sort="seconds")),
            ('xgb', XGBRegressor())])

In [16]:
y_stable_flag = label['stable_flag']
x = pd.DataFrame(index = y_stable_flag.index)

In [17]:
pipeline.set_params(augmenter__timeseries_container=feats_join)
pipeline.fit(x,y_stable_flag)

Feature Extraction: 100%|██████████| 10/10 [27:21<00:00, 164.18s/it]
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/tsfresh/utilities/dataframe_functions.py:172: RuntimeWarning: The columns ['s/CP__fft_coefficient__attr_"real"__coeff_31'
 's/CP__fft_coefficient__attr_"real"__coeff_32'
 's/CP__fft_coefficient__attr_"real"__coeff_33' ...
 's/VS1__fft_coefficient__attr_"angle"__coeff_97'
 's/VS1__fft_coefficient__attr_"angle"__coeff_98'
 's/VS1__fft_coefficient__attr_"angle"__coeff_99'] did not have any finite values. Filling with zeros.
  df.iloc[:, np.where(is_col_non_finite)[0]].columns.values), RuntimeWarning)
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  dat

[19:13:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
         steps=[('augmenter',
                 RelevantFeatureAugmenter(chunksize=None, column_id='cycle',
                                          column_kind=None,
                                          column_sort='seconds',
                                          column_value=None,
                                          default_fc_parameters=None,
                                          disable_progressbar=False,
                                          fdr_level=0.05,
                                          filter_only_tsfresh_features=True,
                                          hypotheses_independent=False,
                                          kind_to_fc_parameters=None,
                                          ml_task='auto', n_jobs=2,
                                          profile=Fals...
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0,
             

In [92]:
type(x)

pandas.core.frame.DataFrame

In [94]:
xtest = pd.DataFrame(index = np.arange(2500,2600,10))

In [95]:
pipeline.predict(xtest)

AttributeError: The ids of the time series container and the index of the input data X do not share any identifier!

In [96]:
feats_join

,seconds,cycle,s/CP,s/CE,s/EPS1,s/FS1,s/FS2,s/PS1,s/PS2,s/PS3,s/PS4,s/PS5,s/PS6,s/SE,s/TS1,s/TS2,s/TS3,s/TS4,s/VS1
0,0,0,2.184,47.202,2411.6,8.990,10.179,151.47,125.500,2.305,0.000,9.936,9.818,68.039,35.570,40.961,38.320,30.363,0.604
1,1,0,2.184,47.273,2411.6,0.770,10.174,151.45,125.390,2.305,0.000,9.947,9.823,0.000,35.492,40.949,38.332,30.375,0.605
2,2,0,2.184,47.250,2411.6,0.641,10.151,151.52,125.400,2.336,0.000,9.964,9.844,0.000,35.469,40.965,38.320,30.367,0.611
3,3,0,2.185,47.332,2411.6,0.006,10.149,151.27,125.030,2.578,0.000,9.989,9.861,0.000,35.422,40.922,38.324,30.367,0.603
4,4,0,2.178,47.213,2411.6,0.000,10.172,150.80,124.050,2.977,0.000,9.996,9.877,0.000,35.414,40.879,38.332,30.379,0.608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132295,55,2204,2.134,46.355,2652.0,0.001,10.179,186.38,0.562,0.000,10.235,9.999,9.873,68.167,35.441,40.910,38.195,30.395,0.516
132296,56,2204,2.146,46.432,2667.0,0.000,10.183,187.20,0.555,0.000,10.198,9.976,9.861,68.167,35.437,40.895,38.184,30.391,0.528
132297,57,2204,2.144,46.384,2690.8,0.000,10.198,187.80,0.484,0.000,10.191,9.960,9.837,68.258,35.434,40.883,38.184,30.395,0.522
132298,58,2204,2.136,46.479,2698.4,0.003,10.190,188.05,0.453,0.000,10.189,9.955,9.827,68.258,35.434,40.879,38.184,30.402,0.522
